In [1]:
import yaml

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
with open('../config.yml') as f:
    config = yaml.load(f)

In [3]:
youtube = build('youtube', 'v3', developerKey=config['DEVELOPER_KEY'])

In [4]:
response = youtube.search().list(
    q='webdriver torso',
    type='video',
    part='id,snippet',
    maxResults=3
).execute()

In [5]:
response

{u'etag': u'"Y3xTLFF3RLtHXX85JBgzzgp2Enw/gOU-G_Pw39VY42hfNrZV30g0OFk"',
 u'items': [{u'etag': u'"Y3xTLFF3RLtHXX85JBgzzgp2Enw/SfbLWwfKQp49skBA3AcsRRsHxbI"',
   u'id': {u'kind': u'youtube#video', u'videoId': u'CMWFPHN2RB8'},
   u'kind': u'youtube#searchResult',
   u'snippet': {u'channelId': u'UCS7-srn4oSGubaIalO2vdPA',
    u'channelTitle': u'ElementNiine',
    u'description': u'Updated description: Since people still believe that this over 1 year "mystery" isn\'t solved, I\'ll paste some of BBC\'s article about Webdriver Torso here. The proof ...',
    u'liveBroadcastContent': u'none',
    u'publishedAt': u'2014-05-21T12:43:46.000Z',
    u'thumbnails': {u'default': {u'url': u'https://i.ytimg.com/vi/CMWFPHN2RB8/default.jpg'},
     u'high': {u'url': u'https://i.ytimg.com/vi/CMWFPHN2RB8/hqdefault.jpg'},
     u'medium': {u'url': u'https://i.ytimg.com/vi/CMWFPHN2RB8/mqdefault.jpg'}},
    u'title': u'Webdriver Torso Mystery - Solved'}},
  {u'etag': u'"Y3xTLFF3RLtHXX85JBgzzgp2Enw/qi9V-C7b41Ht03

In [6]:
# Serialize the structure with PyYAML
with open("cache_test.yml", 'w') as f:
    f.write(yaml.dump(response))

In [7]:
# Deserialize the same structure
with open("cache_test.yml") as f:
    deserialized = yaml.load(f)

In [8]:
# For deep equality testing of the original and deserialized structures:
#   https://gist.github.com/samuraisam/901117

#Copyright (c) 2010-2013 Samuel Sutch [samuel.sutch@gmail.com]
#
#Permission is hereby granted, free of charge, to any person obtaining a copy
#of this software and associated documentation files (the "Software"), to deal
#in the Software without restriction, including without limitation the rights
#to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
#copies of the Software, and to permit persons to whom the Software is
#furnished to do so, subject to the following conditions:
#
#The above copyright notice and this permission notice shall be included in
#all copies or substantial portions of the Software.

import datetime, time, functools, operator, types

default_fudge = datetime.timedelta(seconds=0, microseconds=0, days=0)

def deep_eq(_v1, _v2, datetime_fudge=default_fudge, _assert=False):
  _deep_eq = functools.partial(deep_eq, datetime_fudge=datetime_fudge, 
                               _assert=_assert)
  
  def _check_assert(R, a, b, reason=''):
    if _assert and not R:
      assert 0, "an assertion has failed in deep_eq (%s) %s != %s" % (
        reason, str(a), str(b))
    return R
  
  def _deep_dict_eq(d1, d2):
    k1, k2 = (sorted(d1.keys()), sorted(d2.keys()))
    if k1 != k2: # keys should be exactly equal
      return _check_assert(False, k1, k2, "keys")
    
    return _check_assert(operator.eq(sum(_deep_eq(d1[k], d2[k]) 
                                       for k in k1), 
                                     len(k1)), d1, d2, "dictionaries")
  
  def _deep_iter_eq(l1, l2):
    if len(l1) != len(l2):
      return _check_assert(False, l1, l2, "lengths")
    return _check_assert(operator.eq(sum(_deep_eq(v1, v2) 
                                      for v1, v2 in zip(l1, l2)), 
                                     len(l1)), l1, l2, "iterables")
  
  def op(a, b):
    _op = operator.eq
    if type(a) == datetime.datetime and type(b) == datetime.datetime:
      s = datetime_fudge.seconds
      t1, t2 = (time.mktime(a.timetuple()), time.mktime(b.timetuple()))
      l = t1 - t2
      l = -l if l > 0 else l
      return _check_assert((-s if s > 0 else s) <= l, a, b, "dates")
    return _check_assert(_op(a, b), a, b, "values")
 
  c1, c2 = (_v1, _v2)
  
  # guard against strings because they are iterable and their
  # elements yield iterables infinitely. 
  # I N C E P T I O N
  for t in types.StringTypes:
    if isinstance(_v1, t):
      break
  else:
    if isinstance(_v1, types.DictType):
      op = _deep_dict_eq
    else:
      try:
        c1, c2 = (list(iter(_v1)), list(iter(_v2)))
      except TypeError:
        c1, c2 = _v1, _v2
      else:
        op = _deep_iter_eq
  
  return op(c1, c2)

In [9]:
# Now do a deep comparison to prove that caching results with
# PyYAML won't mess things up
deep_eq(response, deserialized)

True